In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats import power, proportion
import pickle
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import row
from bokeh.models import ColorBar, LogTicker, LogColorMapper
from bokeh import palettes
output_notebook()
%matplotlib inline

In [ ]:
pwr = power.NormalIndPower()

In [ ]:
# compute the effect size and compute sample size for every combination of p1 and p2
n = 101
p1 = np.linspace(0, 1, n)
p2 = np.linspace(0, 1, n)
nobs = np.zeros((n, n)) + np.nan

for i, p1_ in enumerate(p1):
    for j, p2_ in enumerate(p2):
        h = proportion.proportion_effectsize(p1_, p2_)
        if i == j:
            continue
        try:
            nobs[i, j] = pwr.solve_power(
                effect_size=h, alpha=0.05, power=0.8, 
                ratio=1.0, alternative='two-sided'
            )
        except:
            pass

In [ ]:
# define the colormap
inferno256 = palettes.inferno(256)
mn, mx = np.nanmin(np.log10(nobs)), np.nanmax(np.log10(nobs))
cmap = np.linspace(mn, mx, 256)

In [ ]:
# assign color, xy coordinates, difference in p and behavior change
color = []
x, y, dp, bc = [], [], [], []
for i, p1_ in enumerate(p1):
    for j, p2_ in enumerate(p2):
        z = np.log10(nobs[i, j])
        if np.isnan(z):
            c = '#ffffff'
        else:
            idx = (z >= cmap).sum()-1
            c = inferno256[idx]
        
        if p1_ > 0:
            bc_ = (p2_ - p1_) / p1_
        else:
            bc_ = np.nan
        
        color.append(c)
        x.append(p1_)
        y.append(p2_)
        dp.append(p2_ - p1_)
        bc.append(bc_)

In [ ]:
# create data source for plot
src=dict(
    xname=x,
    yname=y,
    dp=dp,
    bc=bc,
    colors=color,
    count=nobs.flatten()
)

tooltips = [
    ('Control', '@xname{0.00}'), 
    ('Test', '@yname{0.00}'), 
    ('Test - Control', '@dp{0.00}'),
    ('Behavior Change', '@bc{0.00}'),
    ('Sample Size', '@count{0}')
]

p = figure(
    title="Minimum Sample Size",
    tools="hover,crosshair",
    x_range=(0, 1), y_range=(0, 1),
    tooltips = tooltips
)

p.plot_width = 800
p.plot_height = 600
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "12pt"
p.xaxis.axis_label = 'Control Probability'
p.yaxis.axis_label = 'Test Probability'


p.rect('xname', 'yname', 0.01, 0.01, source=src,
       color='colors', line_color='colors',
       hover_line_color='black', hover_color='colors')

color_mapper = LogColorMapper(palette="Inferno256", low=10**mn, high=10**mx)

# add colorbar with log scale
color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0))

p.add_layout(color_bar, 'right')

show(p) # show the plot